In [1]:
import pandas as pd
import os

def unir_csv_scrapeados(csv_files, output_dir="../data/merge_data", output_filename="noticias_unidas.csv"):
    """
    Une múltiples archivos CSV de noticias scrapeadas en un solo archivo
    
    Args:
        csv_files: lista de rutas de archivos CSV a unir
        output_dir: directorio donde guardar el archivo unido
        output_filename: nombre del archivo CSV resultante
    
    Returns:
        str: ruta del archivo CSV resultante
    """
    
    # Crear directorio de salida si no existe
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    dataframes = []
    
    print("Cargando archivos CSV...")
    for csv_file in csv_files:
        if os.path.exists(csv_file):
            df = pd.read_csv(csv_file, encoding='utf-8')
            print(f"✓ {os.path.basename(csv_file)}: {len(df)} registros")
            dataframes.append(df)
        else:
            print(f"✗ Archivo no encontrado: {csv_file}")
    
    if not dataframes:
        print("No se encontraron archivos CSV válidos")
        return None
    
    # Unir todos los dataframes
    df_unido = pd.concat(dataframes, ignore_index=True)
    
    # Ruta del archivo de salida
    output_path = os.path.join(output_dir, output_filename)
    
    # Guardar archivo unido
    df_unido.to_csv(output_path, index=False, encoding='utf-8')
    
    print(f"\n{'='*50}")
    print(f"RESUMEN DE LA UNIÓN")
    print(f"{'='*50}")
    print(f"Total de registros: {len(df_unido)}")
    print(f"Distribución por fuente:")
    print(df_unido['fuente'].value_counts())
    print(f"\nDistribución por veracidad:")
    veracidad_counts = df_unido['veracidad'].value_counts()
    print(f"Verdaderas (1): {veracidad_counts.get(1, 0)}")
    print(f"Falsas (0): {veracidad_counts.get(0, 0)}")
    print(f"\nArchivo guardado en: {output_path}")
    
    return output_path

In [2]:
# Lista de archivos CSV a unir
archivos_csv = [
    "../data_scrapeada/noticiasColombiaCheck.csv",
    "../data_scrapeada/noticias_bbc.csv"
]

# Unir los archivos CSV
archivo_resultado = unir_csv_scrapeados(
    csv_files=archivos_csv,
    output_dir="../data/merge_data",
    output_filename="noticias_combinadas.csv"
)

Cargando archivos CSV...
✓ noticiasColombiaCheck.csv: 75 registros
✓ noticias_bbc.csv: 75 registros

RESUMEN DE LA UNIÓN
Total de registros: 150
Distribución por fuente:
fuente
colombiaCheck    75
bbc              75
Name: count, dtype: int64

Distribución por veracidad:
Verdaderas (1): 75
Falsas (0): 75

Archivo guardado en: ../data/merge_data\noticias_combinadas.csv


In [3]:
# Análisis adicional del dataset unido
if archivo_resultado and os.path.exists(archivo_resultado):
    print("\n" + "="*60)
    print("ANÁLISIS DETALLADO DEL DATASET UNIDO")
    print("="*60)
    
    # Cargar el dataset unido
    df_final = pd.read_csv(archivo_resultado, encoding='utf-8')
    
    print(f"Dimensiones del dataset: {df_final.shape}")
    print(f"Columnas: {list(df_final.columns)}")
    
    print(f"\nInformación por fuente:")
    for fuente in df_final['fuente'].unique():
        subset = df_final[df_final['fuente'] == fuente]
        verdaderas = len(subset[subset['veracidad'] == 1])
        falsas = len(subset[subset['veracidad'] == 0])
        print(f"  {fuente}: {len(subset)} total ({verdaderas} verdaderas, {falsas} falsas)")
    
    print(f"\nPrimeros 3 registros:")
    print("-" * 40)
    for i, row in df_final.head(3).iterrows():
        print(f"Fuente: {row['fuente']}")
        print(f"Título: {row['titulo'][:80]}{'...' if len(row['titulo']) > 80 else ''}")
        print(f"Veracidad: {'Verdadera' if row['veracidad'] == 1 else 'Falsa'}")
        print("-" * 40)
    
    print(f"\nDuplicados por título: {df_final['titulo'].duplicated().sum()}")
    print(f"Valores nulos por columna:")
    print(df_final.isnull().sum())
else:
    print("No se pudo crear o encontrar el archivo unido")


ANÁLISIS DETALLADO DEL DATASET UNIDO
Dimensiones del dataset: (150, 5)
Columnas: ['fuente', 'titulo', 'descripcion', 'url', 'veracidad']

Información por fuente:
  colombiaCheck: 75 total (0 verdaderas, 75 falsas)
  bbc: 75 total (75 verdaderas, 0 falsas)

Primeros 3 registros:
----------------------------------------
Fuente: colombiaCheck
Título: Busto del expresidente Nieto en alocución de Petro fue reemplazado por el de Sta...
Veracidad: Falsa
----------------------------------------
Fuente: colombiaCheck
Título: Video de pesqueros chinos circula como si fuera la Flotilla Global Sumud rumbo a...
Veracidad: Falsa
----------------------------------------
Fuente: colombiaCheck
Título: Sanciones de la JEP a exjefes de las FARC por secuestro sí deben reparar y ser m...
Veracidad: Falsa
----------------------------------------

Duplicados por título: 0
Valores nulos por columna:
fuente         0
titulo         0
descripcion    0
url            0
veracidad      0
dtype: int64


In [4]:
# Verificar que el archivo se guardó en la ubicación correcta
expected_path = "../data/merge_data/noticias_combinadas.csv"
full_path = os.path.abspath(expected_path)

print("VERIFICACIÓN FINAL:")
print(f"Ruta esperada: {expected_path}")
print(f"Ruta completa: {full_path}")
print(f"Archivo existe: {os.path.exists(expected_path)}")

if os.path.exists(expected_path):
    file_size = os.path.getsize(expected_path)
    print(f"Tamaño del archivo: {file_size:,} bytes ({file_size/1024/1024:.2f} MB)")
    
    # Verificar que el directorio coincide con la estructura del proyecto
    merge_dir = os.path.dirname(full_path)
    print(f"Directorio de destino: {merge_dir}")
    
    # Mostrar archivos en el directorio merge_data
    if os.path.exists(merge_dir):
        print(f"Archivos en {os.path.basename(merge_dir)}:")
        for file in os.listdir(merge_dir):
            if file.endswith('.csv'):
                file_path = os.path.join(merge_dir, file)
                size = os.path.getsize(file_path)
                print(f"  - {file} ({size:,} bytes)")
else:
    print("❌ El archivo no se creó correctamente")

VERIFICACIÓN FINAL:
Ruta esperada: ../data/merge_data/noticias_combinadas.csv
Ruta completa: c:\Users\ABRAHAM\Documents\GitHub\Practica-1\data\merge_data\noticias_combinadas.csv
Archivo existe: True
Tamaño del archivo: 767,200 bytes (0.73 MB)
Directorio de destino: c:\Users\ABRAHAM\Documents\GitHub\Practica-1\data\merge_data
Archivos en merge_data:
  - full_dataset.csv (5,611,844 bytes)
  - noticias_combinadas.csv (767,200 bytes)
